<a href="https://colab.research.google.com/github/nitron-alpha-kplr/Machine-Learning/blob/main/prediction_diabetes_pyspark_mllib_exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TACHE 1 : Installation des dépendances et exécution de la session Spark

In [1]:
#installer PySpark
!pip install pyspark
from pyspark.ml.classification import LogisticRegression

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=ae0dc4f3300511189ba535f6986e1b6f8c1af0643de293f10ebb79ae5c7319a5
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [8]:
#créer une session Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

# TACHE 2: Cloner et explorer le jeu de données

In [9]:
#cloner le jeu de données sur le diabète depuis le dépôt github
!git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 13.00 KiB | 4.33 MiB/s, done.


In [20]:
#vérifier si le jeu de données existe
!#fill here diabetes_dataset
!ls diabetes_dataset




diabetes.csv  new_test.csv


In [21]:
#créer un dataframe spark
df = spark.read.csv('./diabetes_dataset/diabetes.csv', header=True, inferSchema=True)


In [24]:
#afficher le dataframe
#fill here
df.show()


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [23]:
#afficher le schéma
#fill here
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [25]:
#afficher le nombre de lignes et le nombre de colonnes du jeu de données
#fill here
from pprint import pprint 
pprint(df.columns)

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']


In [27]:
import numpy as np
df.count()

2000

In [29]:
#compter le nombre total de diabétiques et de non-diabétiques
df.groupby('Outcome').count()#fill here.#fill here
print((df.count(),len(df.columns)))

(2000, 9)


In [30]:
#obtenir les statistiques sommaires
#fill here
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

# TACHE 3: Nettoyage et préparation des données

In [31]:
#vérification des valeurs nulles
for col in df.columns:
  print(col+ ":", df[df[col].isNull()].count())
  #check for null values


Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [32]:
#rechercher les valeurs inutiles présentes
def count_zeros():
  column_list= ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
  for i in column_list:
    print(i+":",df[df[i]==0].count())

In [ ]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [33]:
#calculer et remplacer les valeurs inutiles par la valeur moyenne
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data = df.agg({i:'mean'}).first()[0]
  print("mean value for {} is {}".format(i,int(data)))
  df = df.withColumn(i,when(df[i]==0,int(data)).otherwise(df[i]))

mean value for Glucose is 121
mean value for BloodPressure is 69
mean value for SkinThickness is 20
mean value for Insulin is 80
mean value for BMI is 32


In [34]:
#afficher le dataframe
#fill here
#display the dataframe 
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


# TACHE 4: Analyse de corrélation et sélection des caractéristiques

In [36]:
#trouver la corrélation entre l'ensemble des variables d'entrée et de sortie
#find the correlation among the set of input & output variables
for i in df.columns:
  print("Correlation to outcome for  {} is {}".format(i,df.stat.corr('Outcome',i)))

Correlation to outcome for  Pregnancies is 0.22443699263363961
Correlation to outcome for  Glucose is 0.48796646527321064
Correlation to outcome for  BloodPressure is 0.17171333286446713
Correlation to outcome for  SkinThickness is 0.1659010662889893
Correlation to outcome for  Insulin is 0.1711763270226193
Correlation to outcome for  BMI is 0.2827927569760082
Correlation to outcome for  DiabetesPedigreeFunction is 0.1554590791569403
Correlation to outcome for  Age is 0.23650924717620253
Correlation to outcome for  Outcome is 1.0


In [37]:
#sélection des caractéristiques
#nous choisirons toutes les caractéristiques d'entrée de notre modèle comme étant des co_relation similaires
from pyspark.ml.feature import  VectorAssembler
#définition d'une nouvelle colonne de caractéristiques
assembler = VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'], outputCol='features')
output_data = assembler.transform(df)#fill here(df) # produce a new column 'features

In [38]:
#afficher le schéma
#fill here
#print the schema
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [39]:
#afficher le dataframe
#fill here
#display dataframe
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

# TACHE 5: Diviser l'ensemble de données et construire le modèle

In [40]:
#create final data
#nous allons maintenant utiliser le modèle de l'algorithme de régression logistique
from pyspark.ml.classification import LogisticRegression
# Sélection des caractéristiques et de la colonne des résultats
final_data = output_data.select('features','Outcome')

In [41]:
#afficher le schéma de final data
#fill here
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [42]:
#diviser l'ensemble de données 
train ,test=final_data.randomSplit([0.7,0.3])
models = LogisticRegression(labelCol='Outcome')
model = models.fit(train)

In [44]:
#entraîner le modèle
model = models.fit(train)#fill here(train)


In [48]:
#résumé du modèle
#summary of the model
summary = model.summary


In [49]:
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|            Outcome|         prediction|
+-------+-------------------+-------------------+
|  count|               1392|               1392|
|   mean| 0.3390804597701149| 0.2607758620689655|
| stddev|0.47356732595604645|0.43921566102131215|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



# TACHE 6: Évaluer et sauvegarder le modèle

In [50]:
# Par défaut, il utilise l'aire sous le ROC comme matrice de performance.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.evaluate(test)


In [51]:
predictions.predictions.show(10)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,67.0,76.0,20...|      0|[2.31538657236764...|[0.91014335658543...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.51559837914336...|[0.97112834666533...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.46218307833109...|[0.92144782327399...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.46218307833109...|[0.92144782327399...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.46218307833109...|[0.92144782327399...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.70817238743092...|[0.93750715897249...|       0.0|
|[0.0,91.0,68.0,32...|      0|[2.19435555903185...|[0.89974149186409...|       0.0|
|[0.0,91.0,80.0,20...|      0|[2.39188645331271...|[0.91620650892181...|       0.0|
|[0.0,93.0,60.0,25...|      0|[2.72153622479806...|[0.93828554994410...|    

In [52]:
#en utilisant l'évaluateur de classification binaire
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.8396226415094347

In [53]:
#sauvegarder le modèle
#fill here("model")
# save model
model.save(("model"))

In [54]:
#charger le modèle sauvegardé dans l'environnement
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load('model')

# TACHE 7: Prédiction sur de nouvelles données avec le modèle sauvegardé

In [55]:
#créer un nouveau dataframe Spark
test_df = spark.read.csv("/content/diabetes_dataset/new_test.csv", header=True, inferSchema=True)

In [56]:
#afficher le schéma
test_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)



In [57]:
#créer une colonne fusionnée supplémentaire
test_data = assembler.transform(test_df)

In [58]:
#afficher le schéma
#fill here
#print the schema
test_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#utiliser le modèle pour faire des prédictions
results = model.transform(#fill here)
results.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [61]:
#afficher les prévisions
#display the predictions
results.select('features','prediction').show()

NameError: ignored